In [ ]:
import pandas as pd

df = pd.read_excel('/content/encoded_allstats.xlsx')

columns_to_remove = ['+/-', 'OFFRTG', 'DEFRTG', 'NETRTG', 'PIE', 'PTS', 'FGM', 'FTM', '3PM', 'FGA', 'FTA', '3PA']

df = df.drop(columns_to_remove, axis=1)

In [ ]:
# import necessary libraries
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
# Make the train-test split in 80:20 ratio
X = df.drop('W/L', axis = 1)
y = df['W/L']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

# Normalize the X_train and X_test datasets by the min and max values
# scaler = MinMaxScaler()
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Initialize the SVM Classifier
model = SVC(kernel='linear')  # You can choose other kernels like 'rbf', 'poly', etc.

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8434959349593496


***Apply K-Fold CV ***

In [ ]:
from sklearn.model_selection import cross_val_score
kfold = cross_val_score(model, X, y, cv=5)
kfold

array([0.96666667, 0.96666667, 0.93333333, 0.93333333, 1.        ])

In [ ]:
kfold.mean()

0.96

**Apply Stratified K-fold CV**

In [ ]:
from sklearn.model_selection import StratifiedKFold
strat_kfold = cross_val_score(model, X, y, cv=StratifiedKFold())
print(strat_kfold)

[0.96666667 0.96666667 0.93333333 0.93333333 1.        ]


In [ ]:
strat_kfold.mean()

0.96

**Apply LOOCV**

In [ ]:
from sklearn.model_selection import LeaveOneOut
scores2 = cross_val_score(rfc, X, y, cv=LeaveOneOut())
print(scores2)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.]


In [ ]:
scores2.mean()

0.96

**Hyperparameter Tuning**

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter ranges to be tested
C_range = [0.1, 1, 10, 100]              # Regularization parameter
kernel_range = ['linear', 'rbf', 'poly'] # Kernel functions to be tested
gamma_range = ['scale', 'auto']          # Kernel coefficient for 'rbf' and 'poly'

# Create the parameter grid
param_grid = {
    'C': C_range,
    'kernel': kernel_range,
    'gamma': gamma_range
}

# Create the GridSearchCV object
grid = GridSearchCV(estimator=SVC(),
                    param_grid=param_grid,
                    cv=5,
                    scoring='accuracy',
                    refit=True,  # Setting grid with estimator
                    verbose=1)

# Fit the grid
grid.fit(X_train, y_train)

# Print the best parameters and best score
print("Accuracy: ", grid.best_score_)
print(grid.best_params_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Accuracy:  0.8536495266142262
{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}


# **RANDOM FOREST**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train,y_train)
y_pred = rfc.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7886178861788617


**Hyperparameter Tuning**

In [ ]:
from sklearn.model_selection import GridSearchCV

criteria = ["gini", "entropy"]              #criteria to be tested
min_sample_split_range = [2,10, 20]         #min sample split to be tested
max_depth_range = [None, 2, 5, 10]          #max depth to be tested
min_samples_leaf_range = [1, 5, 10]         #min samples in the leaf to be tested
min_leaf_nodes_range = [None, 5, 10, 20]    #min leaf nodes to be tested
# Added parameters
n_estimators = [100, 300, 600]              #n_estimators to be tested#

param_grid = {"criterion": criteria,
              "min_samples_split": min_sample_split_range,
              "max_depth": max_depth_range,
              "min_samples_leaf": min_samples_leaf_range,
              #"max_leaf_nodes": min_leaf_nodes_range,
              "n_estimators" : n_estimators
                }

grid = GridSearchCV(estimator=RandomForestClassifier(),
                    param_grid=param_grid,
                    cv = 5,
                    scoring='accuracy',
                    refit=True, #setting grid with estimator
                    verbose=1)

grid.fit(X_train, y_train)
print("Accuracy:", grid.best_score_)
print(grid.best_params_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Accuracy: 0.8114710479068986
{'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


# **Naive Bayes**

In [ ]:
from sklearn.naive_bayes import GaussianNB
gaussian = GaussianNB()
gaussian.fit(X_train, y_train)
y_pred = gaussian.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7235772357723578


**Hyperparameter Tuning**

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter ranges to be tested
var_smoothing_range = [1e-9, 1e-7, 1e-5, 1e-3, 1e-1]

# Create the parameter grid
param_grid = {
    'var_smoothing': var_smoothing_range
}

# Create the GridSearchCV object
grid = GridSearchCV(estimator=GaussianNB(),
                    param_grid=param_grid,
                    cv=5,
                    scoring='accuracy',
                    refit=True,  # Setting grid with estimator
                    verbose=1)

# Fit the grid
grid.fit(X_train, y_train)

# Print the best parameters and best score
print("Accuracy: ", grid.best_score_)
print(grid.best_params_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Accuracy:  0.7565918807558674
{'var_smoothing': 1e-09}
